# Конкурентное  программирование

## 1. Конкурентное != параллельное

Мы говорим о многозадачности, то есть о выполнении нескольких задач в один и тот же период времени. С многозадачностью связывают конкурентное, параллельное и асинхронное выполнение задач.

**Конкурентные задачи** (англ. concurrent tasks) — задачи, выполнение которых пересекается во времени. Задачи могут выполняться параллельно, а могут порционно (виртуальный параллелизм).

**Параллельные задачи** (англ. parallel tasks) — задачи, которые буквально выполняются одновременно.

Из определений видно, что конкурентность и параллелизм различаются. Это две независимые характеристики, которые образуют 4 вида многозадачности.
![Concurrency/parallelism types](img/parallel_concurrent_table.png)

На рисунке ниже показано различие между последовательным, конкурентным и параллельным выполнением задач. Круги — этапы вычислений, а стрелки показывают последовательность вычислений.

![Concurrency vs parallelism vs sequantial](img/parallel_sequential_concurrent.jpg)

**Асинхронные задачи** (англ. asynchronous tasks) — задачи, которые запущены без ожидания результата. Таким образом, асинхронная задача не блокирует запускающую систему. Обычно асинхронные задачи по окончанию вызывают функцию обратного вызова (англ. callback function).

В противоположность асинхронным задачам ставят синхронные. На рисунке ниже показано время синхронного и асинхронного выполнения задач.

![Sync/async types](img/synchronous-asynchronous.png)

Организовать выполение задач конкурентно можно двумя сопособами:
- решение с общей памятью (англ. shared memory),
- решение с передачей сообщений (англ. message passing).

Оба подхода могут быть использованы на машинах с разной архитектурой. Но если задачи выполняются на машине с физически разделеённой памятью, то проще огранизовать конкурентую работу на передаче сообщений.

## 2. Подход с общей памятью

В этом подходе конкурентные задачи взаимодействуют через общии участки памяти. Такая система может быть построена на разных сущностях:
- процессы,
- потоки,
- кооперативные потоки: «зелёные потоки», протопотоки, файберы и корутины.

### 2.1. Синхронизация

#### Критическая секция и «гонки»
При этом доступ к общему ресурсу могут возникнуть «гонки» (англ. race condition) — проблема, при которой работа системы или приложения зависит от того, в каком порядке выполняются конкурентные части программы. Эти части, которые могут приводить к «гонкам», называют критическими секциями (англ. critical section). Перед входом в критическую секцию конкурентные задачи надо синхронизовать. 

![Race condition](img/knock_knock.jpg)

Различают следующие классические примитивные способы синхронизации:

- Семафор
    * Семафор со счётчиком (англ. counting semaphore)
    * Бинарный семафор (англ. binary semaphore), он жe lock
        - RWLock (=Readers-Writes lock)
    * Событие (англ. event)
        - Условная переменная (англ. conditional variable)
- Мьютекс (англ. mutex = mutual exclusion)
    * Рекурсивный мьютекс (англ. recursive mutex)
    * Shared-мьютекс (shared mutex)
    * Спин-блокировка (англ. spin lock)
    * Фьютекс (англ. futex = fast userspace mutex)
- Монитор (англ. monitor)
- Барьер aka Рандеву

**NB!** Избежать синхронизации можно разработкой программы без блокировок.

### 2.2. Processes

Процесс (анг. process) — загруженная в память программа. Процессы относятся к основному механизму многозадачности операционных систем. Процессы запускаются конкурентно и переключение между процессами (англ. scheduling) контролируется операционной системой. Она делает это по таймеру. Такую схему преключения называют вытесняющей многозадачностью.

#### Время жизни процессов в Unix. Низкоуровневая работа с процессами

Посмотреть список процессов в unix-подобной операционной системе можно следующей командой.

In [100]:
!ps xo 'pid ppid stat comm'

  PID  PPID STAT COMMAND
    1     0 Ss   sh
    7     1 Sl   jupyter-noteboo
   17     7 Ssl  python
  144    17 R    python
  209    17 Ss+  sh
  210   209 R+   ps


Здесь `PID` — идентификатор процесса, `PPID` — идентификатор родительского процесса. В unix-системах состояние процессов представляет собой дерево, где в корне находится процесс под номером 1.

Мы можем создать новый процесс только через клонирование другого процесса. Это делается системным вызовом `fork`. Этим вызовом создётся виртуальное адресное пространство нового процесса и ему присваевается `PID`. Однако, явного копирования памяти родительского процесса в дочерний не происходит. Только если в родительском, либо в дочернем процессе выполняется запись в определённое адресное простраство, то оно копируется. Этот механизм называется copy-on-write.

Вызвать `fork` можно с помощью модуля `os`.

In [68]:
import os

child = os.fork() # возвращает pid дочернего процесса
print(f'pid: {child}')

pid: 144
pid: 0


Видно, что `print` выполнился 2 раза. Один раз код был выполнен в child-процессе, а другой — в parent. В child-процессе  `os.fork` возвращает 0, а в parent-процессе — `PID` дочернего процесса. Проверкой на 0 можно понять где мы запущены.

Дочерний процесс должен появиться в списке процессов системы с соответвующим `PID`.

In [99]:
!ps xo 'pid ppid stat comm'

  PID  PPID STAT COMMAND
    1     0 Ss   sh
    7     1 Sl   jupyter-noteboo
   17     7 Ssl  python
  144    17 R    python
  207    17 Ss+  sh
  208   207 R+   ps


Общение между приложениями и ядром операционной системы выполняется через два механизма:
- системные вызовы,
- сигналы.

![System calls and signals](img/syscall_signal.png)

Посмотрим несколько первых сигналов, поддерживаемых в системе

In [54]:
!kill -l | head

0
HUP
INT
QUIT
ILL
TRAP
ABRT
BUS
FPE
KILL


In [102]:
import signal

os.kill(child, signal.SIGKILL)

In [103]:
!ps xo 'pid ppid stat comm'

  PID  PPID STAT COMMAND
    1     0 Ss   sh
    7     1 Sl   jupyter-noteboo
   17     7 Rsl  python
  144    17 Z    python <defunct>
  211    17 Ss+  sh
  212   211 R+   ps


`Z` — зомби-процесс, он как бы мёртв, но ядро хранит код возврата и `PID`.

In [104]:
# Читаем код возврата
os.waitpid(0, 0)

(144, 9)

In [120]:
!ps xo 'pid ppid stat comm'

  PID  PPID STAT COMMAND
    1     0 Ss   sh
    7     1 Sl   jupyter-noteboo
   17     7 Ssl  python
  235    17 Z    python <defunct>
  237    17 Ss+  sh
  238   237 R+   ps


#### Работа с процессами в Python

In [106]:
from multiprocessing import Process

def fun_in_child():
    print('Hello from child!')
    
p = Process(target=fun_in_child)
p.start()
p.join()

Hello from child!


In [127]:
!ps xo 'pid ppid stat comm'

  PID  PPID STAT COMMAND
    1     0 Ss   sh
    7     1 Sl   jupyter-noteboo
   17     7 Rsl  python
  249    17 Ss+  sh
  250   249 R+   ps


#### Межпроцессное взаимодействие
Для параллельной работы процессы используют механизмы межпроцессного взаимодействия (англ. inter-process communication, IPC):
- Сокеты: Unix-сокеты, Беркли-сокеты, Windows-сокеты *linux, windows, mac*
- Анонимный канал (англ. pipe) *linux, windows*
- Именованный канал (англ. fifo) *linux*
- Сегменты общей памяти (shared memory segments) *linux*
- Очереди сообщений (англ. message queues) *mac, linux*
- Сигналы (англ. signals) *linux*
- Отображение файлов (англ. file mapping) *linux, windows, mac*
- Удаленный вызов процедур (англ. Remote Procedure Call) *windows, mac*
- Шина dbus *linux*
- Система FUSE *linux*
- Mailslots *windows*
- Буфер обмена (англ. clipboard) *windows*
- Система Dynamic Data Exchange *windows*


![Linux IPC](img/ipc_linux.png)

In [126]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


### 2.3. Threads

Поток (анг. thread) — подзадача процесса. Потоки ещё называют «лёгкими процессами» (англ. light weight process). Переключение между потоками также контролируются операционной системой. Здесь также присутсвует вытесняющая многозадачность — переключение по таймеру. Все потоки одного процесса обладают общей виртуальной памятью.

Посмотреть список потоков и процессов в unix-подобной операционной системе можно следующей командой. Мы добавили `-T` к предыдущей.

In [33]:
!ps -Tef

UID        PID  SPID  PPID  C STIME TTY          TIME CMD
root         1     1     0  0 21:51 ?        00:00:00 /bin/sh -c pipenv run jupy
root         6     6     1  0 21:51 ?        00:00:04 /usr/local/bin/python /usr
root         6    36     1  0 21:52 ?        00:00:00 /usr/local/bin/python /usr
root         6    37     1  0 21:52 ?        00:00:00 /usr/local/bin/python /usr
root        35    35     6  0 21:52 ?        00:00:02 /usr/local/bin/python -m i
root        35    41     6  0 21:52 ?        00:00:00 /usr/local/bin/python -m i
root        35    42     6  0 21:52 ?        00:00:00 /usr/local/bin/python -m i
root        35    43     6  0 21:52 ?        00:00:00 /usr/local/bin/python -m i
root        35    44     6  0 21:52 ?        00:00:00 /usr/local/bin/python -m i
root        35    45     6  0 21:52 ?        00:00:00 /usr/local/bin/python -m i
root        35    46     6  0 21:52 ?        00:00:00 /usr/local/bin/python -m i
root        35    47     6  0 21:52 ?        00:00:

Здесь `SPID` — идентификатор потока.

In [1]:
from threading import Thread

In [22]:
def subproc(n: int):
    [print(i) for i in range(n)]
    
thread1 = Thread(target=subproc, args=(5,))
thread2 = Thread(target=subproc, args=(5,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


In [40]:
class MyThread(Thread):
    def __init__(self, n:int):
        super().__init__(name=f"Up to {n}")
        self.n = n
    def run(self):
        [print(i) for i in range(self.n)]
        
thread1 = MyThread(5)
thread2 = MyThread(5)

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


#### Демонизация потоков

Процесс будет работать до тех пор, пока все потоки не будут завершены. Если надо по явному завершению процесса прекратить работу всех дочерних потоков, то их надо сделать демонами.

In [31]:
def subproc(n: int):
    [print(i) for i in range(n)]
    
thread1 = Thread(target=subproc, args=(5,), daemon=True)
thread2 = Thread(target=subproc, args=(5,), daemon=True)

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


#### GIL (Global Interpreter Lock)

GIL — глобальный lock интерпретатора. Это особенность реализации Python из-за которой нельзя одновременно использовать несколько процессоров для потоков. Все параллельные вычисления в Python реализуются с помощью модулей.

#### Мьютекс

In [41]:
from threading import Lock

In [47]:
mutex = Lock()

mutex.acquire()
print(mutex)
# Работа с общими ресурсами
mutex.release()
print(mutex)

<locked _thread.lock object at 0x109b52a80>
<unlocked _thread.lock object at 0x109b52a80>


In [64]:
x = 0
def fun_inc(n:int):
    global x
    for _ in range(n):
        x += 1
    
def fun_dec(n:int):
    global x
    for _ in range(n):
        x -= 1

thread1 = Thread(target=fun_inc, args=(5000,))
thread2 = Thread(target=fun_dec, args=(5000,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

print(x)

0


In [76]:
try:
    mutex.acquire()
    # Работа с общими ресурсами

except:
    # Обрабатываем исключения
    pass

finally:
    mutex.release()

#### Рекурсивный мьютекс

In [74]:
from threading import RLock

#### Семафор

In [67]:
from threading import Semaphore

In [69]:
s = Semaphore(10)

s.acquire()
print(s)
# Работа с общими ресурсами
s.release()
print(s)

#### События

In [70]:
from threading import Event

In [73]:
e = Event()

#e.wait() # Ждём, когда кто-нибудь захватит флаг
print(e)
e.set() # Ставим флаг
print(e)
# Работа с общими ресурсами
e.clear() # Снимаем флаг и ждём нового
print(e)

### 2.4. Кооперативные потоки: green threads, protothreads, fibers и coroutines

Эти разновидности потоков объединяет то, что операционная система о таких потоках ничего не знает. Они эмулируются приложением. Вместо вытесняющей многозадачности они управляются кооперативной многозадачностью — поток сам явно объявляет, когда он готов отдать процессорное время другому такому же потоку. Из-за этого такие потоки легче: быстрее создаются и между ними выполняется быстрое переключение.


## 3. Подход с передачей сообщений

Модель передачи сообщений. Существет стандартный интерфейс передачи сообщений (англ. Message Passing Interface, MPI).

- Асинхронная передача сообщений. Actor model
- Синхронная передача сообщений. Communicating sequential processes

## 4. Имеет ли смысл распараллеливать задачу?

**Закон Амдала** (англ. Amdahl’s Law)
Закон Амдала показывает во сколько раз меньше времени потребуется параллельной программе для решения задачи по сравнению с последовательной. 
$$
\text{Speed-up}=\frac{1}{S+P/n}
$$

## 5. Аппаратный взгляд на паралелизм

### Классификация Флина

[Классификация Флина](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)

![Flynn's Taxonomy](img/parallel-architectures.png)

## 6. Почитать
- [(Stackoverflow) Processes, threads, green threads, protothreads, fibers, coroutines: what's the difference?](https://stackoverflow.com/questions/3324643/processes-threads-green-threads-protothreads-fibers-coroutines-whats-the)
- [Такие удивительные семафоры](https://habr.com/en/post/261273/)
- [Как устроен GIL в Python](https://habr.com/en/post/84629/)
- [Parallel Computing vs. Distributed Computing: A Great Confusion?](https://link.springer.com/chapter/10.1007%2F978-3-319-27308-2_4)